In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected

# mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
# shapes = pd.read_csv('./data/vectors_1.csv')
# shapes = pd.read_csv('./data/vectors_pillow.csv')
shapes = pd.read_csv('./data/vectors_birdhouse.csv')
# shapes = pd.read_csv('./data/vectors_washingmachine.csv')
# remove the first index col
shapes = shapes.iloc[0:39,:]

shapes = shapes.iloc[:,1:]
print(shapes.shape)
# print(shapes)

(39, 6162)


In [26]:
# Params

num_inputs = 6162    #28x28 pixels
num_hid1 = 1000
num_hid2 = 10
num_hid3 = num_hid1
num_output = num_inputs
lr = 0.01
actf = tf.nn.relu


In [27]:
# Create Layers 

X = tf.placeholder(tf.float32,shape=[None,num_inputs])
initializer = tf.variance_scaling_initializer()

w1=tf.Variable(initializer([num_inputs,num_hid1]),dtype=tf.float32)
w2=tf.Variable(initializer([num_hid1,num_hid2]),dtype=tf.float32)
w3=tf.Variable(initializer([num_hid2,num_hid3]),dtype=tf.float32)
w4=tf.Variable(initializer([num_hid3,num_output]),dtype=tf.float32)

b1=tf.Variable(tf.zeros(num_hid1))
b2=tf.Variable(tf.zeros(num_hid2))
b3=tf.Variable(tf.zeros(num_hid3))
b4=tf.Variable(tf.zeros(num_output))

hid_layer1=actf(tf.matmul(X,w1) + b1)
hid_layer2=actf(tf.matmul(hid_layer1,w2) + b2)
hid_layer3=actf(tf.matmul(hid_layer2,w3) + b3)
output_layer=actf(tf.matmul(hid_layer3,w4) + b4)

loss = tf.reduce_mean(tf.square(output_layer-X))

optimizer = tf.train.AdamOptimizer(lr)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [ ]:
num_epoch=50
batch_size=150
num_test_images=10
num_train = shapes.shape[0]

loss_epoch_log = []

builder = tf.saved_model.builder.SavedModelBuilder('./tmp/saved_model_birdhouse')

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epoch):
        
        # num_batches=mnist.train.num_examples//batch_size
        
#         num_batches=num_train//batch_size
#         for iteration in range(num_batches):
#             # X_batch,y_batch=mnist.train.next_batch(batch_size)
#             X_batch = shapes
#             sess.run(train,feed_dict={X:X_batch})
            
        sess.run(train,feed_dict={X:shapes})
            
        train_loss=loss.eval(feed_dict={X:shapes})
        print("epoch {} loss {}".format(epoch,train_loss))
        
        loss_epoch_log.append({ 'epoch': epoch, 'train_loss': train_loss })
        
    results=output_layer.eval(feed_dict={X:shapes}, session=sess)
    print('results')
        
    print(results)
    # Show loss graph
#     loss_epoch_log = pd.DataFrame(loss_epoch_log)
#     plt.scatter(loss_epoch_log.epoch, loss_epoch_log.train_loss)
#     plt.show()
    
    # Creating signatures
    tensor_info_x = tf.saved_model.utils.build_tensor_info(X)
    tensor_info_hid2 = tf.saved_model.utils.build_tensor_info(hid_layer2)
    tensor_info_output_layer = tf.saved_model.utils.build_tensor_info(output_layer)
    
    encode_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
              inputs={'input_vector': tensor_info_x},
              outputs={'code': tensor_info_hid2},
              method_name='encode'))
    
    decode_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
              inputs={'code': tensor_info_hid2},
              outputs={'output_vector': tensor_info_output_layer},
              method_name='encode'))
    
    # Saving the model
    builder.add_meta_graph_and_variables(
        sess, [tf.saved_model.tag_constants.SERVING], 
        signature_def_map={
            'encode': encode_signature,
            'decode': decode_signature
        },
        strip_default_attrs=True)
    builder.save()
    

    # Save the model
#     saver = tf.train.Saver()
#     save_path = saver.save(sess, "./tmp/model.ckpt")
#     print('saved at', save_path)
    
    
#     tf.saved_model.simple_save(sess,
#             './tmp/simple_save',
#             inputs={"X": shapes},
#             outputs={"output_layer": output_layer })
#     print('saved with simple save')
    
    print('w1', w1.eval())
    
    # Do Random Inputs
    vec = output_layer.eval(feed_dict={hid_layer2: np.random.rand(100,10)}, session=sess)
    df = pd.DataFrame(vec)
    df.to_csv('./data/results_random_2.csv')
    # print(df)
    
    

epoch 0 loss 114455453696.0
epoch 1 loss 324452.25
epoch 2 loss 324451.90625
epoch 3 loss 324450.4375
epoch 4 loss 324447.5
epoch 5 loss 324442.84375
epoch 6 loss 324436.59375
epoch 7 loss 324428.1875
epoch 8 loss 324417.875
epoch 9 loss 324405.28125
epoch 10 loss 324390.4375
epoch 11 loss 324372.96875
epoch 12 loss 324352.9375
epoch 13 loss 324330.25
epoch 14 loss 324304.6875
epoch 15 loss 324276.15625
epoch 16 loss 324244.59375
epoch 17 loss 324209.8125
epoch 18 loss 324171.75
epoch 19 loss 324130.21875
epoch 20 loss 324085.09375
epoch 21 loss 324036.40625
epoch 22 loss 323983.96875


In [24]:
# Encode
saver = tf.train.Saver()

with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    # vec = sess.run(hid_layer2, feed_dict={X:shapes})
    # print(vec.shape)
    # df = pd.DataFrame(vec)
    # df.to_csv('./data/pillow_encoded.csv')
    # print(df)
    
    vec = output_layer.eval(feed_dict={hid_layer2: np.random.rand(100,10)}, session=sess)
    df = pd.DataFrame(vec)
    df.to_csv('./data/results_random_2.csv')
    
    
# Decode

ValueError: The passed save_path is not a valid checkpoint: /tmp/model.ckpt